<a href="https://colab.research.google.com/github/italolima04/monitoria-probabilidade-estatistica/blob/master/Pesquisa-PID/Implementa%C3%A7%C3%A3o-Algoritmos/Word2VEC_DadosTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Bibliotecas

**Notebook produzido com base no Artigo: Unsupervised Sentiment Analysis de Rafał Wójcik e no tutorial: Gensim Word2Vec Tutorial de Pierre Megret**

In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import spacy  
import nltk
nltk.download('stopwords')
import re  
from collections import defaultdict 
import logging 
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Dados do Artigo Referenciado

**Verificando estrutura do Dataset utilizado no Estudo referenciado.**

In [207]:
#Criando DataFrame a partir dos dados em formato CSV.
data = pd.read_csv('polish_sentiment_dataset.csv')

In [208]:
#Visualizando as 5 primeiras linhas do DataFrame.
data.head()

,description,length,rate
0,Polecam nie pierwszy i nie ostatni raz!,39.0,1.0
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,121.0,1.0
2,Polecam zakupy w tym sklepie. Są dostępne częś...,87.0,1.0
3,0,0.0,0.0
4,Jestem w pełni zadowolona z przebiegu transakcji,48.0,1.0


In [209]:
#Verificando as 5 últimas linhas do DataFrame.
data.tail()

,description,length,rate
936878,Coraz lepiej wygląda,NaN,-1.0
936879,JA SRAM NA TEN PIERSCIONEK I NA CIEBIE CHWILE ...,NaN,-1.0
936880,Rafatus do Marleny Ty kurwo bez honoru ...,NaN,-1.0
936881,matka Marleny prosi o pomoc,NaN,-1.0
936882,wiesz człowieku że on ją nawet nie uderzył i m...,NaN,-1.0


**Pode-se observar que apesar de não serem dados rotulados, para cada frase há uma taxa relacionada.**

In [210]:
#Descrevendo os dados a partir de medidas estatísticas.
data.describe()

,length,rate
count,762836.000000,936817.000000
mean,88.486888,0.587340
std,97.483536,0.797016
min,0.000000,-1.000000
25%,44.000000,1.000000
50%,63.000000,1.000000
75%,101.000000,1.000000
max,7970.000000,1.000000


In [211]:
#Verificando a ausência dos dados para as colunas do DataFrame.
data.isna().sum()

description       629
length         174047
rate               66
dtype: int64

In [212]:
#Verificando a distribuição da taxa atribuída a cada frase, dentre a coleção de frases.
data['rate'].value_counts()

 1.0    734250
-1.0    184020
 0.0     18547
Name: rate, dtype: int64

# Estudo com a Base de Dados do Twitter

In [52]:
#Criando DataFrame a partir dos dados coletados do Twitter, que se encontram dispostos em formato CSV.
df = pd.read_csv('dados-pesquisa.csv')
#Verificando as 10 últimas linhas do DataFrame.
df.tail(10)

,Unnamed: 0,created_at,id,text,user,lang
109954,109954,Mon Jul 20 15:08:32 +0000 2020,1.285230e+18,@oatila Então bastam distanciamento social e m...,"{'id': 1057053859774902273, 'id_str': '1057053...",pt
109955,109955,Mon Jul 20 15:07:52 +0000 2020,1.285230e+18,sim isso mesmo peçam lockdown pro meu pai que ...,"{'id': 2939855297, 'id_str': '2939855297', 'na...",pt
109956,109956,Mon Jul 20 15:07:52 +0000 2020,1.285230e+18,@SissyRibs @MichaelAncap @_XURIQUEM_ @izaqueba...,"{'id': 1143970643207184384, 'id_str': '1143970...",pt
109957,109957,Mon Jul 20 15:05:08 +0000 2020,1.285229e+18,"Sem frear aumento de casos da Covid, cidades d...","{'id': 60722362, 'id_str': '60722362', 'name':...",pt
109958,109958,Mon Jul 20 15:04:28 +0000 2020,1.285229e+18,Serasse eu fiz esse post sobre quarentena por ...,"{'id': 3019776221, 'id_str': '3019776221', 'na...",pt
109959,109959,Mon Jul 20 15:04:14 +0000 2020,1.285229e+18,porto alegre perigando a entrar em lockdown e ...,"{'id': 1245313171, 'id_str': '1245313171', 'na...",pt
109960,109960,Mon Jul 20 15:03:35 +0000 2020,1.285229e+18,"Nunca que a Mídia vai falar isso!\nAMB, CFM e ...","{'id': 40697641, 'id_str': '40697641', 'name':...",pt
109961,109961,Mon Jul 20 15:03:33 +0000 2020,1.285229e+18,@joaopiresrj Eu não sou cientista mas esse LOC...,"{'id': 942176977, 'id_str': '942176977', 'name...",pt
109962,109962,Mon Jul 20 15:01:54 +0000 2020,1.285228e+18,Q&amp;A - Recessão ou Lockdown. O que é pior? ...,"{'id': 1136982345137958912, 'id_str': '1136982...",pt
109963,109963,Mon Jul 20 15:01:38 +0000 2020,1.285228e+18,O prefeito de Los Angeles está preste a decret...,"{'id': 494366716, 'id_str': '494366716', 'name...",pt


In [53]:
#Filtrando os dados e selecionando somente as colunas de interesse inicial para o estudo.
df = df[['created_at', 'text']]
#Visualizando as 5 últimas linhas dos dados.
df.tail()

,created_at,text
109959,Mon Jul 20 15:04:14 +0000 2020,porto alegre perigando a entrar em lockdown e ...
109960,Mon Jul 20 15:03:35 +0000 2020,"Nunca que a Mídia vai falar isso!\nAMB, CFM e ..."
109961,Mon Jul 20 15:03:33 +0000 2020,@joaopiresrj Eu não sou cientista mas esse LOC...
109962,Mon Jul 20 15:01:54 +0000 2020,Q&amp;A - Recessão ou Lockdown. O que é pior? ...
109963,Mon Jul 20 15:01:38 +0000 2020,O prefeito de Los Angeles está preste a decret...


In [54]:
#Verificando a estrutura do DataFrame.
df.shape

(109964, 2)

Observa-se a presença de aproximadamente 110k de linhas, cujas representam os tweets filtrados.

In [55]:
#Verificando a existência de dados nulos.
df.isnull().sum()

created_at    0
text          0
dtype: int64

In [56]:
#Removendo dados duplicados na coluna de Texto e substituindo dentro do próprio Data Frame.
df.drop_duplicates(['text'], inplace=True)

In [57]:
#Verificando novamente a estrutura dos dados para observar a quantidade de dados únicos.
df.shape

(103364, 2)

Cerca de 6.6k de Tweets foram removidos, devido ao fato de que o conteúdo se encontrava duplicado na base de dados.

# Pré-Processando os Dados

**Função que remove links, urls, sinais de pontuação, padroniza os caracteres como minúsculos e remove as stopwords através de um dos módulos da biblioteca NLTK.**

In [60]:
def Preprocessing_data(instance):
      instance = re.sub(r"http\S+", "", instance).lower().replace('.', '').replace(';','').replace('-','').replace(':', '').replace(')', '')
      stopwords = set(nltk.corpus.stopwords.words('portuguese'))
      words = [i for i in instance.split() if not i in stopwords]
      return (" ".join(words))

In [59]:
#Utilizando notação comum em estudos de Aprendizado de Máquina.
X = df['text']

In [61]:
#Aplicando Função aos dados de Texto.
X = [Preprocessing_data(i) for i in X]

In [62]:
#Visualizando os dados após a aplicação da função de Pré-Processamento.
X[:15]

['tudo mudou forma trabalhar, nada mudou objetivo informar #jornalismo #imprensa…',
 'mcdonald’s reabriu lojas público #comida #covid19 #mcdonalds',
 'sindicato, atento tudo envolve saúde condições trabalho nessa pandemia #covid @spbancarios',
 'problemas saúde definitivamente acabaram #saude #forabolsonaro #covid',
 'fábricas todo mundo viram obrigadas dispensarem funcionários decorrência #covid19 antes m…',
 '@g1 @redeglobo @jornalhoje governo @jairbolsonaro vai socorrer ninguém vai enrolar! pois minto quer r…',
 'lei n° 6666, #eua, pretende traçar investigar percurso pessoas tivestes contatos, usand…',
 'governo precisa ser responsabilizado #covid #covid19',
 'blockchain tecnologia popularizou #bitcoin sendo testada combate #covid19 sistema capaz de…',
 'conhecem algum país além brasil torcida organizada coronavirus???? #covid #covid19',
 'itajuípe recebe centro municipal isolamento covid19 #covid19 #isolamentosocial #covid @rctitajuipe',
 '📣 notícia / news / nouvelles 🇵🇹 manual boa

In [69]:
#Criando DataFrame com os dados pré-processados.
data_text = pd.DataFrame(data=X, columns=['text'])

In [70]:
#Visualizando DataFrame.
data_text

,text
0,"tudo mudou forma trabalhar, nada mudou objetiv..."
1,mcdonald’s reabriu lojas público #comida #covi...
2,"sindicato, atento tudo envolve saúde condições..."
3,problemas saúde definitivamente acabaram #saud...
4,fábricas todo mundo viram obrigadas dispensare...
...,...
103359,porto alegre perigando entrar lockdown ônibus ...
103360,"nunca mídia vai falar isso! amb, cfm crms, tod..."
103361,@joaopiresrj cientista lockdown acho q duas se...
103362,q&ampa recessão lockdown pior? | fiique tranqu...


# Transformando os Dados

In [16]:
#Importando módulos do Gensim, biblioteca que auxilia na implementação do Modelo Word2VEC.
from gensim.models.phrases import Phrases, Phraser

INFO - 14:01:10: 'pattern' package not found; tag filters are not available for English


In [71]:
#Quebrando as frases por palavra.
sent = [row.split() for row in data_text['text']]

In [72]:
#Visualizando a representação criada acima.
sent

[['tudo',
  'mudou',
  'forma',
  'trabalhar,',
  'nada',
  'mudou',
  'objetivo',
  'informar',
  '#jornalismo',
  '#imprensa…'],
 ['mcdonald’s',
  'reabriu',
  'lojas',
  'público',
  '#comida',
  '#covid19',
  '#mcdonalds'],
 ['sindicato,',
  'atento',
  'tudo',
  'envolve',
  'saúde',
  'condições',
  'trabalho',
  'nessa',
  'pandemia',
  '#covid',
  '@spbancarios'],
 ['problemas',
  'saúde',
  'definitivamente',
  'acabaram',
  '#saude',
  '#forabolsonaro',
  '#covid'],
 ['fábricas',
  'todo',
  'mundo',
  'viram',
  'obrigadas',
  'dispensarem',
  'funcionários',
  'decorrência',
  '#covid19',
  'antes',
  'm…'],
 ['@g1',
  '@redeglobo',
  '@jornalhoje',
  'governo',
  '@jairbolsonaro',
  'vai',
  'socorrer',
  'ninguém',
  'vai',
  'enrolar!',
  'pois',
  'minto',
  'quer',
  'r…'],
 ['lei',
  'n°',
  '6666,',
  '#eua,',
  'pretende',
  'traçar',
  'investigar',
  'percurso',
  'pessoas',
  'tivestes',
  'contatos,',
  'usand…'],
 ['governo', 'precisa', 'ser', 'responsabilizado

In [73]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

INFO - 14:12:26: collecting all words and their counts
INFO - 14:12:26: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 14:12:27: PROGRESS: at sentence #10000, processed 104026 words and 97237 word types
INFO - 14:12:27: PROGRESS: at sentence #20000, processed 205867 words and 170938 word types
INFO - 14:12:27: PROGRESS: at sentence #30000, processed 306361 words and 238343 word types
INFO - 14:12:27: PROGRESS: at sentence #40000, processed 407874 words and 301827 word types
INFO - 14:12:27: PROGRESS: at sentence #50000, processed 509384 words and 364125 word types
INFO - 14:12:28: PROGRESS: at sentence #60000, processed 610794 words and 431743 word types
INFO - 14:12:28: PROGRESS: at sentence #70000, processed 711725 words and 491391 word types
INFO - 14:12:28: PROGRESS: at sentence #80000, processed 810918 words and 547391 word types
INFO - 14:12:28: PROGRESS: at sentence #90000, processed 911020 words and 600180 word types
INFO - 14:12:28: PROGRESS: at sentence #

In [74]:
#Construindo modelo baseado em Bigram, para a detecção de palavras (únicas) que são formadas por outras duas.
bigram = Phraser(phrases)

INFO - 14:12:28: source_vocab length 674495
INFO - 14:12:34: Phraser built with 861 phrasegrams


In [75]:
sentences = bigram[sent]

In [76]:
#Percorrendo os dados e fazendo uma contagem para verificar as palavras mais frequentes da base.
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

117576

In [83]:
#Exibindo as 10 palavras mais frequentes da base de dados.
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['coronavírus',
 'covid19',
 'lockdown',
 'coronavirus',
 'brasil',
 'pra',
 'casos',
 'sobre',
 'mortes',
 'contra']

# Criando o Modelo Word2VEC.

In [185]:
#Importando biblioteca e módulo do Gensim, para a implementação do Word2VEC.
import multiprocessing

from gensim.models import Word2Vec

In [186]:
#Função que conta o número de cores da máquina.
cores = multiprocessing.cpu_count()

In [187]:
#Instanciando modelo Word2VEC a partir dos mesmos parâmetros estabelecidos pelo artigo.
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     size=300,
                     sample=0.8, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

In [188]:
#Construindo vocabulário a partir das sentenças.
w2v_model.build_vocab(sentences, progress_per=10000)

INFO - 14:50:08: collecting all words and their counts
INFO - 14:50:08: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 14:50:08: PROGRESS: at sentence #10000, processed 98386 words, keeping 26703 word types
INFO - 14:50:09: PROGRESS: at sentence #20000, processed 194307 words, keeping 41176 word types
INFO - 14:50:09: PROGRESS: at sentence #30000, processed 289323 words, keeping 53198 word types
INFO - 14:50:09: PROGRESS: at sentence #40000, processed 384343 words, keeping 63822 word types
INFO - 14:50:10: PROGRESS: at sentence #50000, processed 480594 words, keeping 73460 word types
INFO - 14:50:10: PROGRESS: at sentence #60000, processed 577296 words, keeping 82906 word types
INFO - 14:50:10: PROGRESS: at sentence #70000, processed 672253 words, keeping 91879 word types
INFO - 14:50:11: PROGRESS: at sentence #80000, processed 765282 words, keeping 100227 word types
INFO - 14:50:11: PROGRESS: at sentence #90000, processed 858298 words, keeping 107673 word typ

# Treinando o Modelo Word2VEC

In [189]:
#Treiando modelo Word2VEC.
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

INFO - 14:50:18: training model with 1 workers on 30785 vocabulary and 300 features, using sg=0 hs=0 sample=0.8 negative=20 window=4
INFO - 14:50:19: EPOCH 1 - PROGRESS: at 8.80% examples, 73340 words/s, in_qsize 1, out_qsize 0
INFO - 14:50:20: EPOCH 1 - PROGRESS: at 17.90% examples, 75978 words/s, in_qsize 1, out_qsize 0
INFO - 14:50:21: EPOCH 1 - PROGRESS: at 27.07% examples, 76975 words/s, in_qsize 1, out_qsize 0
INFO - 14:50:22: EPOCH 1 - PROGRESS: at 36.27% examples, 77279 words/s, in_qsize 1, out_qsize 0
INFO - 14:50:23: EPOCH 1 - PROGRESS: at 45.23% examples, 74528 words/s, in_qsize 1, out_qsize 0
INFO - 14:50:24: EPOCH 1 - PROGRESS: at 54.26% examples, 75451 words/s, in_qsize 1, out_qsize 0
INFO - 14:50:25: EPOCH 1 - PROGRESS: at 63.40% examples, 76200 words/s, in_qsize 1, out_qsize 0
INFO - 14:50:26: EPOCH 1 - PROGRESS: at 72.65% examples, 76627 words/s, in_qsize 1, out_qsize 0
INFO - 14:50:27: EPOCH 1 - PROGRESS: at 81.94% examples, 77053 words/s, in_qsize 1, out_qsize 0
INFO

(26567790, 29622330)

In [190]:
#Tornando o modelo mais eficiente em questões de uso de memória.
w2v_model.init_sims(replace=True)

INFO - 14:55:34: precomputing L2-norms of word weight vectors


**A partir do Word2VEC podemos verificar a similaridade de outras palavras para determinados termos.**

In [191]:
w2v_model.wv.most_similar(["vacina"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('vacina_contra', 0.609605073928833),
 ('vacina_chinesa', 0.5751551985740662),
 ('tomar_vacina', 0.5486936569213867),
 ('"vacina', 0.5384091138839722),
 ('vacinas', 0.5198544263839722),
 ('doses_vacina', 0.5139744281768799),
 ('cura_pro', 0.493501752614975),
 ('testes_vacina', 0.46525266766548157),
 ('#vacina', 0.45659971237182617),
 ('vacina_oxford', 0.447509229183197)]

**Podemos determinar filtros para uma verificação de similaridade.**

In [192]:
w2v_model.wv.most_similar(["recuperados"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('curados', 0.671785295009613),
 ('pacientes_recuperados', 0.667811930179596),
 ('pessoas_recuperadas', 0.6040254235267639),
 ('casos_ativos', 0.588263988494873),
 ('pessoas_curadas', 0.5835896730422974),
 ('vítimas_fatais', 0.5382181406021118),
 ('recuperados…', 0.531403124332428),
 ('confirmações', 0.5296151041984558),
 ('mortes_confirmadas', 0.5178470015525818),
 ('pessoas_infectadas', 0.5082339644432068)]

In [193]:
w2v_model.wv.most_similar(["mortes"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('mortes_causadas', 0.7485598921775818),
 ('novas_mortes', 0.701056957244873),
 ('mortes_confirmadas', 0.6430712938308716),
 ('mortes,', 0.6371706128120422),
 ('vidas_perdidas', 0.6332045197486877),
 ('óbitos', 0.6250780820846558),
 ('1300_mortes', 0.6233320236206055),
 ('mortos', 0.6231629252433777),
 ('mortes_diárias', 0.6212472915649414),
 ('mil_mortes', 0.6204636096954346)]

**Podemos compararar a taxa de similaridade para termos específicos.**

In [194]:
w2v_model.wv.similarity("coronavirus", 'mortes')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-0.3836797

In [195]:
w2v_model.wv.similarity("coronavirus", 'casos')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-0.40394896

In [196]:
w2v_model.wv.similarity("pandemia", 'coronavirus')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-0.10929663

# Aplicando K-Means

In [197]:
#Importando algoritmo KMeans da biblioteca Sklearn.
from sklearn.cluster import KMeans

In [198]:
word_vectors = w2v_model.wv
model = KMeans(n_clusters=3, init='k-means++', max_iter=1000, n_init=50).fit(X=word_vectors.vectors)
positive_cluster_center = model.cluster_centers_[0]
negative_cluster_center = model.cluster_centers_[1]
neutral_cluster_center = model.cluster_centers_[2]

In [199]:
word_vectors.vectors

array([[-0.0081975 ,  0.04720695, -0.04763551, ...,  0.01966804,
         0.11293161, -0.07009899],
       [ 0.0133413 ,  0.10912789, -0.05271741, ...,  0.03482201,
         0.09564679, -0.10163205],
       [-0.01201384, -0.09868426,  0.04223893, ..., -0.05724829,
        -0.06662759, -0.02191113],
       ...,
       [-0.03365825, -0.00169525,  0.06411723, ...,  0.0335671 ,
        -0.02276869,  0.06044143],
       [ 0.0180357 ,  0.01752128, -0.02925419, ...,  0.02912579,
         0.00297402,  0.00167207],
       [ 0.04025285,  0.03373059, -0.02376372, ..., -0.02053193,
        -0.00142729,  0.04017974]], dtype=float32)

In [200]:
model.cluster_centers_

array([[-2.69353129e-02, -1.28714060e-02,  1.18289161e-02,
        -8.39804113e-03, -1.28686344e-02,  4.41465527e-05,
        -2.90056523e-02,  1.15854386e-02, -1.67149282e-03,
        -3.71328294e-02, -1.38816666e-02, -3.42764296e-02,
         1.55072119e-02,  1.80993397e-02,  2.20879875e-02,
         4.55052918e-03,  4.11116984e-04,  1.23869535e-03,
         2.72744056e-03, -8.67928937e-03, -2.24724039e-02,
        -3.66220810e-02,  2.15306431e-02,  4.53119725e-03,
        -8.47364124e-03, -2.32212003e-02,  1.71815231e-02,
        -9.19501763e-03,  1.63319856e-02,  7.27554457e-03,
         1.79597382e-02,  1.65261850e-02, -7.74508249e-03,
        -7.94370379e-03,  2.81177391e-03,  6.44521601e-03,
         1.13219740e-02, -9.63797793e-03, -2.57210294e-03,
        -2.19659992e-02, -1.55437228e-04,  1.39029641e-02,
        -3.06272730e-02, -9.10585467e-03,  3.75240557e-02,
        -1.82352811e-02,  1.06848043e-03,  1.18571417e-02,
        -2.60140672e-02,  3.09641995e-02,  2.81891972e-0

In [201]:
len(model.labels_)

30785

In [202]:
model.labels_

array([1, 1, 0, ..., 2, 0, 1], dtype=int32)

In [203]:
df_kmeans = pd.DataFrame(data=model.labels_, columns=['text'])

In [204]:
df_kmeans

,text
0,1
1,1
2,0
3,0
4,0
...,...
30780,1
30781,2
30782,2
30783,0


In [205]:
df_kmeans['text'].value_counts()

0    11152
2    10041
1     9592
Name: text, dtype: int64

In [206]:
df_kmeans['text'].unique()

array([1, 0, 2], dtype=int32)

# Aplicando Mean Shift

In [213]:
#Importando algoritmo da Biblioteca Sklearn.
from sklearn.cluster import MeanShift

In [ ]:
#Criando instância do algoritmo.
word_vectors = w2v_model.wv
clustering = MeanShift(bandwidth=2).fit(X=word_vectors.vectors)

In [ ]:
#Verificando os labels originados.
clustering.labels_

In [ ]:
df_meanshift = pd.DataFrame(data=model.labels_, columns=['text'])

In [ ]:
df_meanshift

,text
0,1
1,1
2,0
3,0
4,0
...,...
30780,1
30781,2
30782,2
30783,0


In [ ]:
df_meanshift['text'].value_counts()

0    11152
2    10041
1     9592
Name: text, dtype: int64

In [ ]:
df_meanshift['text'].unique()

array([1, 0, 2], dtype=int32)

# Aplicando DBSCAN

In [ ]:
#Importando algoritmo da Biblioteca Sklearn.
from sklearn.cluster import DBSCAN

In [ ]:
#Criando instância do algoritmo e treinando o modelo com os dados.
clustering = DBSCAN(eps=0.5, min_samples=2).fit(X)

In [ ]:
#Verificando os labels originados.
clustering.labels_

array([-1, -1, -1, ..., -1, -1, -1])

# Sumarizando Resultados

In [ ]:
#word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)

In [ ]:
#word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

**t-SNE é um algoritmo de redução de dimensionalidade não linear que tenta representar dados de alta dimensão e as relações subjacentes entre vetores em um espaço de dimensão inferior.**

**Importando Bibliotecas.**

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
def tsnescatterplot(model, word, list_names):
    arrays = np.empty((0, 300), dtype='f')
    word_labels = [word]
    color_list  = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, model.wv.__getitem__([word]), axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar([word])
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)
        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=50).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('t-SNE visualization for {}'.format(word.title()))

In [ ]:
#tsnescatterplot(w2v_model, 'coronavirus', ['pandemia', 'mortes', 'vacina', 'casos', 'virus', 'lockdown'])